In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
       
from HPIB.HP4155 import HP4155
from HPIB.HPT import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P
from HPIB.DevParams import UMC
from HPIB.INOSerial import Arduino

from BFmodule import DR

# from YFunc import YFuncExtraction

from IPython.display import clear_output, display
from os import makedirs, rename
from time import sleep
from datetime import datetime, timedelta

HP=HP4155("GPIB0::17", debug=False)
# HP.reset()
# HP.IntTime="LONG"

INO=Arduino("COM3")
print(INO.ask('*'.encode()))

# Elsa=DR()

HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
b'*'



In [2]:
params = {
'Vmin' : 0,
'Vmax' : 1.5,
'Vstep' : 0.02,

'Vgmin' : 0.6,
'Vgmax' : 1.4,
'Vgstep' : 0.2,

'Vfmin' : 0,
'Vfmax' : 2,
'Vfstep' : 0.01,
'IComp' : 1e-3,

'Ifmin' : -5e-3,
'Ifmax' : 5e-3,
'Ifpoints' : 50,
'Iffactor' : 50,
'VComp' : 1.5,

'Vd' : 0.025,
'temp' : 3.338,
'HiPot' : False,
'min_wait' : 30
}

DeviceList = ['TP2', 'DN1', 'TP12', 'TN4']
# DeviceList = ['', '', '', 'TN4', '', '']
path = "C:/Users/Zucchi/Documents/Medidas/241217 TP2 DN1 TP12 TN4/"
makedirs(path, exist_ok=True)

In [11]:
INO.opench(int(input()))

 2


'Open INO: ch 2'

In [12]:
HP.measure()

0

In [13]:
HP.stop()

0

In [10]:
HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vd'], ptype=True)

Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001


0

In [28]:
pathp=path+'TN3'
makedirs(pathp, exist_ok=True)
now=datetime.now().strftime('%y%m%d %H%M')
        
LIN = PlotVgs(HP.SingleSave(f"{pathp}/IdVgs - {now}.csv", timeout=30))

Done IdxVgs. Duration: 34 s                                     


In [5]:
HP.SetDiode(-1.5, 1.5, 0.05, Comp=0.5e-3)

Set Diode
Vf=(1.5, -1.5)


0

In [11]:
HP.Diode4P(0, 1, 0.01)

Diode=(0, 1, 0.01)


0

In [13]:
HP.SingleDiode(0, 1, 0.05, Comp=0.5e-3, SMUP="SMU2", SMUN="SMU1")

Diode=(0, 1, 0.05)


0

In [24]:
HP.IntTime="LONG"

In [20]:
HP.IntTime="SHOR"

In [30]:
# DeviceList = ['TP1', 'TP8', 'TP5', 'TN10', 'DN2']

for n, device in enumerate(DeviceList):
    ptype=False
    if 'p' in device.lower():
        ptype=True
    pathp=path+device
    
    makedirs(pathp, exist_ok=True)
    plt.close('all')
    if device:
        INO.opench(n+1)

    if 'T' in device.upper():
        HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vd'], ptype=ptype)
        now=datetime.now().strftime('%y%m%d %H%M')
        
        LIN = PlotVgs(HP.SingleSave(f"{pathp}/IdVgs - {now}.csv", timeout=30))
        
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(f"{pathp}/IdVgs - {now}.csv", f"{pathp}/IdVgs - {temp} - {now}.csv")
        rename(f"{pathp}/IdVgs - {now}.png", f"{pathp}/IdVgs - {temp} - {now}.png")
        
        HP.SetDiode(0, 1.5 if ptype else -1.5, 0.05 if ptype else -0.05, Comp=0.5e-3)
        now=datetime.now().strftime('%y%m%d %H%M')
        Plot(HP.SingleSave(f"{pathp}/Diode - {now}.csv", timeout=30), 'Vf', ['Id', 'Is'])
        
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(f"{pathp}/Diode - {now}.csv", f"{pathp}/Diode - {temp} - {now}.csv")
        rename(f"{pathp}/Diode - {now}.png", f"{pathp}/Diode - {temp} - {now}.png")
        
    if 'D' in device.upper():
        now=datetime.now().strftime('%y%m%d %H%M%S')
        HP.SingleDiode(-1.5, 1.5, 0.1, SMUP='SMU2', SMUN='SMU1', Comp=0.5e-3)
        Plot(HP.SingleSave(f"{pathp}/Diode - {now}.csv", timeout=30), 'Vf', 'If')

        now=datetime.now().strftime('%y%m%d %H%M%S')
        HP.SingleDiode(0, -1.5, 0.01, SMUP='SMU4', SMUN='SMU3', Comp=0.5e-3)
        Plot(HP.SingleSave(f"{pathp}/Diode - {now}.csv", timeout=30), 'Vf', 'If')

Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Done IdxVgs. Duration: 39 s                                     
Set Diode
Vf=(0, -1.5)
Done Diode. Duration: 13 s                                     
Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Done IdxVgs. Duration: 35 s                                     
Set Diode
Vf=(0, -1.5)
Done Diode. Duration: 13 s                                     


C:\Users\Zucchi\Jupyter\HPIB\HPIB\HPT.py:81: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax1=plt.subplots(figsize=(sizex*px,(sizex*480/640)*px))


Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Done IdxVgs. Duration: 35 s                                     
Set Diode
Vf=(0, -1.5)
Done Diode. Duration: 13 s                                     
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Done IdxVgs. Duration: 40 s                                     
Set Diode
Vf=(0, 1.5)
Done Diode. Duration: 13 s                                     
